In [ ]:
# =============================
# Data Extraction Notebook
# Project: Energy Prices in Illinois
# Milestone 1
# =============================

In [ ]:
# Import required libraries
import pandas as pd
import requests
from io import StringIO
import pickle
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -----------------------------
# 1. Define Save Paths
# -----------------------------
BASE_PATH = "/content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois"
RAW_PATH = "/content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/raw"
PROC_PATH = "/content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/processed"
os.makedirs(RAW_PATH, exist_ok=True)
os.makedirs(PROC_PATH, exist_ok=True)

In [ ]:
# -----------------------------
# 2. Define Data Sources
# -----------------------------
ACTUAL_CSV    = os.path.join(RAW_PATH, "actual_energy_price.csv")
DAY_AHEAD_CSV = os.path.join(RAW_PATH, "day_ahead_energy_price.csv")

In [ ]:
# -------------------------------
# 3. Helper Function (read CSV)
# -------------------------------
def load_energy_csv(path, name="data"):
    print(f"\nReading {name} from {path} ...")
    df = pd.read_csv(path)

    cols_lower = {c.lower().strip(): c for c in df.columns}
    date_col  = cols_lower.get("date") or cols_lower.get("datetime") or list(df.columns)[0]
    price_col = cols_lower.get("lmp")  or cols_lower.get("price")
    hub_col   = cols_lower.get("hub")  or cols_lower.get("node") or cols_lower.get("location")

    use_cols = [date_col, price_col, hub_col]
    df = df[use_cols].rename(columns={
        date_col:  "date",
        price_col: "price",
        hub_col:   "hub"
    })

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date", "price"]).sort_values("date").reset_index(drop=True)

    print("Columns:", list(df.columns))
    print(df.head(3))
    return df


In [ ]:
# -------------------------------
# 4. Load Datasets (from CSVs)
# -------------------------------
df_actual    = load_energy_csv(ACTUAL_CSV,    "actual_energy_price.csv")
df_day_ahead = load_energy_csv(DAY_AHEAD_CSV, "day_ahead_energy_price.csv")


Reading actual_energy_price.csv from /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/raw/actual_energy_price.csv ...


/tmp/ipython-input-3777422313.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


Columns: ['date', 'price', 'hub']
        date  price           hub
0 2020-10-19  19.59  ARKANSAS.HUB
1 2020-10-19  19.80  ILLINOIS.HUB
2 2020-10-19  20.11   INDIANA.HUB

Reading day_ahead_energy_price.csv from /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/raw/day_ahead_energy_price.csv ...


/tmp/ipython-input-3777422313.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"], errors="coerce")


Columns: ['date', 'price', 'hub']
        date  price           hub
0 2020-10-19  20.50  ARKANSAS.HUB
1 2020-10-19  20.62  ILLINOIS.HUB
2 2020-10-19  20.98   INDIANA.HUB


In [ ]:
# -----------------------------
# 5. Merge & Add Features
# -----------------------------
if not df_actual.empty and not df_day_ahead.empty:
    df = pd.merge(
        df_actual, df_day_ahead,
        on="date", how="inner", suffixes=("_actual", "_dayahead")
    )

    # Calendar features
    df["day_of_week"] = df["date"].dt.day_name()
    df["hour_of_day"] = df["date"].dt.hour
    df["month"] = df["date"].dt.month

    print("✅ Merged dataset:", df.shape)
else:
    print("❌ One of the datasets is empty.")
    df = pd.DataFrame()

✅ Merged dataset: (2028368, 8)


In [ ]:
# -----------------------------
# 6. Debug & Working Datasets
# -----------------------------
df_debug = df.head(500)      # small subset for quick testing
df_working = df.copy()       # full dataset

In [ ]:
# -----------------------------
# 7. Save Processed Data
# -----------------------------
debug_pickle = os.path.join(PROC_PATH, "energy_debug.pkl")
working_pickle = os.path.join(PROC_PATH, "energy_working.pkl")

df_debug.to_pickle(debug_pickle)
df_working.to_pickle(working_pickle)

print(f"💾 Saved processed debug -> {debug_pickle}")
print(f"💾 Saved processed working -> {working_pickle}")

💾 Saved processed debug -> /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/processed/energy_debug.pkl
💾 Saved processed working -> /content/drive/MyDrive/IE434-DeepDive-EnergyPricesInIllinois/Data/processed/energy_working.pkl
